In [ ]:
"""
NOTES:
    - DraftsActive will be available as soon as the draft is entered regardless
    of fill status.
    - DraftDetail picks 

TODO:
    - A shell of the draft will need to be created at the beginning in order
    to derive the # of picks between selections for each user/pick
        - Add a 'users' and 'draft_entries' df to DraftDetail which pulls the 
        data found in the url_draft url and sliced using ['draft']['users']
        and ['draft']['draft_entries']
        - Will want to wait until the status (i.e. ['draft']['status'] == 'drafting')
        - Need to update DraftsDetail to account for IndexError that occurs
        when creating the main df at the very beginning of the draft when
        'picks' is empty


"""

In [1]:
from os.path import join
import pickle

import pandas as pd
import numpy as np
import getpass

import UD_draft_model.scrapers.scrape_site.scrape_league_data as scrape_site
import UD_draft_model.scrapers.scrape_site.pull_bearer_token as pb
import UD_draft_model.data_processing.prepare_drafts as prepare_drafts
import UD_draft_model.data_processing.add_features as add_features
from UD_draft_model.modeling.model_version import ModelVersion

In [2]:
import getpass

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)

### Variables to change ###
chromedriver_path = "/usr/bin/chromedriver"
try:
    print(username)
except NameError:
    username = input("Enter Underdog username: ")

bearer_token = pb.read_bearer_tokens()[username]
valid_token = pb.test_bearer_token(bearer_token)

if valid_token == False:
    password = getpass.getpass()
    url = "https://underdogfantasy.com/lobby"
    bearer_token = pb.pull_bearer_token(url, chromedriver_path, username, password)

    pb.save_bearer_token(username, bearer_token)

In [4]:
player_vars = [
    'appearance_id', 'player_id', 'position', 'first_name', 'last_name',
    'abbr', 'team_name', 'adp', 'season_projected_points'
]

active_drafts = scrape_site.DraftsActive(bearer_token)

df_active = active_drafts.create_df_active_drafts()

draft_entry_id = df_active['draft_entry_id'].iloc[0]
slate_id = df_active['slate_id'].iloc[0]
scoring_type_id = df_active['scoring_type_id'].iloc[0]
draft_id = [df_active['id'].iloc[0]]
rounds = df_active['rounds'].iloc[0]
refs = scrape_site.ReferenceData(slate_id, scoring_type_id)

draft_detail = scrape_site.DraftsDetail(draft_id, bearer_token)

df_players = refs.create_df_players_master()
df_players = df_players[player_vars]
df_draft = draft_detail.create_df_drafts()
df_entries = draft_detail.create_df_draft_entries()

# url = [url for url in draft_detail.url_drafts.values()][0]
# header = draft_detail.auth_header
# header['authorization'] = bearer_token
# _df = draft_detail.read_in_site_data(url, headers=header)

# df
df_players
df_active

,id,auto_pick_at,clock,contest_style_id,draft_at,draft_entry_id,draft_type,entry_count,entry_role,entry_style_id,pick_count,slate_id,source,source_entry_style_id,status,title,user_auto_pick,user_pick_order,scoring_type_id,rounds
0,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,2023-03-20T01:34:20Z,28800,978b95dd-7c25-467c-83c9-332d90a557a4,2023-03-16T17:27:23Z,6667f354-e6b1-49ed-842f-7e029d5f3920,slow,12,None,473af08f-55a4-5f56-b401-2e6ce8d0e096,41,b84244dc-aa63-4b62-bdd5-8fccd365c074,sit_and_go,None,drafting,None,off,5,ccf300b0-9197-5951-bd96-cba84ad71e86,20


In [5]:
def create_draft_board(df_entries: pd.DataFrame, num_rounds: int) -> pd.DataFrame:
    """
    Creates a df of all user/pick numbers given the entries of a snake draft.

    Parameters
    ----------
    df_entries : pd.DataFrame
        All entries in the draft.
        Note that the draft must be full
    num_rounds : int
        Number of rounds in the draft.

    Returns
    -------
    pd.DataFrame
        Draft board containing each pick number for every entry.
    """

    df_asc = df_entries.sort_values(by='pick_order')
    df_desc = df_entries.sort_values(by='pick_order', ascending=False)

    dfs = []
    for round in range(1, num_rounds + 1):
        if round % 2 == 0:
            df_round = df_desc.copy()
        else:
            df_round = df_asc.copy()

        df_round['round'] = round

        dfs.append(df_round)

    df = pd.concat(dfs).reset_index(drop=True)
    df['number'] = df.index + 1
    df.rename(columns={'id': 'draft_entry_id'}, inplace=True)

    keep_vars = [
        'draft_id', 'draft_entry_id', 'user_id', 'username', 'round', 'number'
    ]

    df = df[keep_vars]

    return df


def update_board(df_board: pd.DataFrame, df_draft: pd.DataFrame) -> pd.DataFrame:
    """
    Updates the draft board with the selections made.

    Parameters
    ----------
    df_board : pd.DataFrame
        Draft board containing each pick number for every entry.
    df_draft : pd.DataFrame
        All draft selections that have been made so far.

    Returns
    -------
    pd.DataFrame
        Draft board that contains the player selection made at each pick.
    """

    df = pd.merge(
        df_board, df_draft, 
        how='left', on=['draft_id', 'draft_entry_id', 'number']
    )

    return df


def add_user_next_pick_number(df_board: pd.DataFrame, draft_entry_id: str
) -> pd.DataFrame:
    """
    Adds the pick number of the next pick of interest for the user using
    the app rather than the user currently drafting. "Next pick of interest" 
    refers to the pick number that follows the user's upcoming pick.

    next_pick_number is used to create features that indicate how each players'
    rank relates to the # of picks between the current pick and the 
    next pick of the user currently drafting. Therefore, probability estimates
    will be from the perspective of this user which isn't relevant to the user
    actually utilizing the model. For example, if the user is drafting from pick 6
    and the draft is currently at the 10th pick, probability estimates for pick
    15 are useless. Instead, estimates for pick 30 would allow the user to 
    start preparing for their next pick.

    IMPORTANT: Using this will result in feature and target distributions
    being different from training (e.g. max picks between will be 48 vs. 24)
    which could throw the predictions off. However, this should be insignificant
    once the draft is within a handful of picks from the user.

    Parameters
    ----------
    df_board : pd.DataFrame
        Draft board containing each pick number for every entry.
    draft_entry_id : str
        Draft entry ID to base the next pick number off of.

    Returns
    -------
    pd.DataFrame
        Draft board with the next pick number based off the draft_entry_id
        passed.
    """

    df_user = df_board.loc[df_board['draft_entry_id'] == draft_entry_id].copy()
    df_user.sort_values(by='number', inplace=True)

    df_user['next_pick_number_2'] = df_user['next_pick_number'].shift(-1)

    rename_vars = {
        'number': 'user_number',
        'next_pick_number': 'user_next_pick_number',
        'next_pick_number_2': 'user_next_pick_number_2'
    }
    keep_vars = ['round', 'number', 'next_pick_number', 'next_pick_number_2']
    df_user = df_user[keep_vars].rename(columns=rename_vars)

    # User's next pick number needs to be named 'next_pick_number' for the
    # add_features function.
    df_all = df_board.rename(columns={'next_pick_number': 'next_pick_number_og'})

    df = pd.merge(df_all, df_user, how='left', on='round')
    df['next_pick_number'] = np.where(
        df['number'] <= df['user_number'], 
        df['user_next_pick_number'],
        df['user_next_pick_number_2']
    )

    df.drop(columns=list(rename_vars.values()), inplace=True)

    return df


def get_current_pick(df_board: pd.DataFrame) -> pd.DataFrame:
    """
    Filters the draft board down to the current pick.

    Parameters
    ----------
    df_board : pd.DataFrame
        Draft board that includes all current selections.

    Returns
    -------
    pd.DataFrame
        Filtered Draft board that only contains the next pick to select
        a player.
    """

    df = df_board.loc[df_board['appearance_id'].isnull()].iloc[0:1]

    return df


def get_avail_players(df_players: pd.DataFrame, df_draft: pd.DataFrame) -> pd.DataFrame:
    """
    Pulls the remaining available players to draft.

    Parameters
    ----------
    df_players : pd.DataFrame
        All players that could/can be selected in the draft.
    df_draft : pd.DataFrame
        All draft selections that have been made so far.

    Returns
    -------
    pd.DataFrame
        All players that have NOT been drafted.
    """

    df = (
        df_players
        .loc[~df_players['appearance_id'].isin(df_draft['appearance_id'])]
        # .iloc[:100]
    )

    return df


def add_avail_players(df_cur_pick: pd.DataFrame, df_avail_players: pd.DataFrame
) -> pd.DataFrame:
    """
    Joins the available players onto the current pick df that is required
    for the model features and prediction.

    Parameters
    ----------
    df_cur_pick : pd.DataFrame
        Filtered Draft board that only contains the next pick to select
        a player.
    df_avail_players : pd.DataFrame
        All players that have NOT been drafted.

    Returns
    -------
    pd.DataFrame
        All players that have NOT been drafted with current pick data.
    """

    left_vars = [
        'draft_id', 'draft_entry_id', 'user_id', 'username', 'round',
        'number', 'next_pick_number'
    ]
    df = pd.merge(df_cur_pick[left_vars], df_avail_players, how='cross')
    df.reset_index(inplace=True, drop=True)

    return df


def load_model(file_path: str) -> ModelVersion:
    """
    Loads the ModelVersion object containing the model to be used for creating
    predictions.

    Parameters
    ----------
    file_path : str
        Path to the ModelVersion object

    Returns
    -------
    ModelVersion
        ModelVersion object with model to be used.
    """
    dbfile = open(file_path, 'rb')
    obj = pickle.load(dbfile)
    dbfile.close()

    return obj


def create_predictions(df: pd.DataFrame, model: ModelVersion) -> np.ndarray:
    """
    Applies the model to the df to create new predictions.

    Parameters
    ----------
    df : pd.DataFrame
        df of all players to create a probability estimate for.
    model : ModelVersion
        ModelVersion obj that contains the model to use for predictions.

    Returns
    -------
    np.ndarray
        1D array containing the probability estimates.
    """

    df_features = df[model.metadata['features']]

    y_pred_prob = model.model.predict_proba(df_features)
    y_pred_prob = y_pred_prob[:,1]

    return y_pred_prob


def merge_prediction(df: pd.DataFrame, pred: np.array, out_col: str) -> pd.DataFrame:
    """
    Merges the model's predicions back onto the full df.

    Parameters
    ----------
    df : pd.DataFrame
        df of all available players to draft.
    pred : np.array
        1D array of probability estimates.
    out_col : str
        Name of the probability estiamte column.

    Returns
    -------
    pd.DataFrame
        df of all available players to draft with the probability estimate
        of being picked added.
    """

    df = df.copy()
    df.reset_index(inplace=True, drop=True)

    pred = pd.DataFrame(pred, columns=[out_col])

    df = pd.concat([df, pred], axis=1)
    df[out_col] = df[out_col].astype(float).round(2)

    return df


MODEL_PATH = (
    "/home/cdelong/Python-Projects/UD-Draft-Model/"
    + "Repo-Work/UD-Draft-Model/data/models"
)
MODEL = "LogisticRegression_v01_v001"

model = load_model(join(MODEL_PATH, MODEL))

df_board = create_draft_board(df_entries, rounds)
df_board = prepare_drafts.add_next_pick_number(df_board, filter_nulls=False)
df_board = add_user_next_pick_number(df_board, draft_entry_id)

df_board = update_board(df_board, df_draft)
df_cur_pick = get_current_pick(df_board)
df_avail_players = get_avail_players(df_players, df_draft)
df_w_players = add_avail_players(df_cur_pick, df_avail_players)

df_w_features = add_features.add_features(df_w_players)

probs = create_predictions(df_w_features, model)
df_w_probs = merge_prediction(df_w_features, probs, 'prob')

df_w_probs

,draft_id,draft_entry_id,user_id,username,round,number,next_pick_number,appearance_id,player_id,position,first_name,last_name,abbr,team_name,adp,season_projected_points,avail_cur_rank_actual,picks_btwn,diff_cur_rank_picks_btwn,ind_rank_btwn,prob
0,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,ab29b25e-a8f5-47d7-a26a-556036cbf82f,9ba0ffe1-5312-4d97-82d1-83c74a13bc7d,TRAVIOLLI2K,4,42,53.0,4846ac23-e1e4-4c85-ad16-1fa888630e36,a0fa2a74-640b-4cc1-a5d8-841ab9557fe0,QB,Joe,Burrow,CIN,Cincinnati Bengals,36.6,340.7,1,11.0,-10.0,1,0.86
1,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,ab29b25e-a8f5-47d7-a26a-556036cbf82f,9ba0ffe1-5312-4d97-82d1-83c74a13bc7d,TRAVIOLLI2K,4,42,53.0,dedd0eb7-3d68-4884-8ca3-888ea09a912d,660a9e72-4fc7-442d-ba62-257dbab39568,TE,T.J.,Hockenson,MIN,Minnesota Vikings,43.3,138.7,2,11.0,-9.0,1,0.85
2,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,ab29b25e-a8f5-47d7-a26a-556036cbf82f,9ba0ffe1-5312-4d97-82d1-83c74a13bc7d,TRAVIOLLI2K,4,42,53.0,01ed82c6-3836-4069-b51a-31e60ade3670,c108c752-92fd-491f-94ed-60b0b3cea7d8,WR,Christian,Watson,GB,Green Bay Packers,44.9,171.6,3,11.0,-8.0,1,0.82
3,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,ab29b25e-a8f5-47d7-a26a-556036cbf82f,9ba0ffe1-5312-4d97-82d1-83c74a13bc7d,TRAVIOLLI2K,4,42,53.0,5201736b-fbee-4e43-a111-4b4195abaa56,6081005f-45de-474b-87a3-2b74cd21f9b9,WR,Jerry,Jeudy,DEN,Denver Broncos,44.9,186.5,4,11.0,-7.0,1,0.80
4,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,ab29b25e-a8f5-47d7-a26a-556036cbf82f,9ba0ffe1-5312-4d97-82d1-83c74a13bc7d,TRAVIOLLI2K,4,42,53.0,a4aeeeea-f25c-46a6-a759-63dce7b60296,9edce99f-fad8-4de1-9e20-435b3d12c310,QB,Lamar,Jackson,BAL,Baltimore Ravens,45.0,348.8,5,11.0,-6.0,1,0.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,ab29b25e-a8f5-47d7-a26a-556036cbf82f,9ba0ffe1-5312-4d97-82d1-83c74a13bc7d,TRAVIOLLI2K,4,42,53.0,774e0c10-3b95-4c28-8c07-b41cea33a63a,5a8dfec6-a864-49ff-b02e-8c64192bce48,TE,Jace,Sternberger,NaN,NaN,-,0.0,1519,11.0,1508.0,0,0.00
1519,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,ab29b25e-a8f5-47d7-a26a-556036cbf82f,9ba0ffe1-5312-4d97-82d1-83c74a13bc7d,TRAVIOLLI2K,4,42,53.0,7759e140-b154-40c9-9f72-b13efafeea5f,3fcdf736-9272-4e4f-bbf7-57269a23da38,RB,Xavier,Jones,NaN,NaN,-,0.0,1520,11.0,1509.0,0,0.00
1520,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,ab29b25e-a8f5-47d7-a26a-556036cbf82f,9ba0ffe1-5312-4d97-82d1-83c74a13bc7d,TRAVIOLLI2K,4,42,53.0,777b7808-b98a-4a72-80e8-6a08e8dc7916,4249eca7-1da3-43d8-ba3c-8aac0f4157f6,WR,Ryan,McDaniel,NaN,NaN,-,0.0,1521,11.0,1510.0,0,0.00
1521,2d30ccdc-4b4a-4d19-9532-d13b1cc33a3b,ab29b25e-a8f5-47d7-a26a-556036cbf82f,9ba0ffe1-5312-4d97-82d1-83c74a13bc7d,TRAVIOLLI2K,4,42,53.0,7789e2a7-8244-4cc7-b523-319691f06153,685aaeb8-9d0b-4823-92cb-bff8cec67076,TE,JJ,Howland,NaN,NaN,-,0.0,1522,11.0,1511.0,0,0.00


In [38]:
out = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model/UD_draft_model/scratch'
active_drafts = scrape_site.DraftsActive(bearer_token)

df = pd.read_pickle(join(out, 'active_drafts2.pkl'))
df = active_drafts._add_scoring_type(df.copy())

slate_id = df['slate_id'].iloc[0]
scoring_type_id = df['scoring_type_id'].iloc[0]
draft_id = [df['id'].iloc[0]]
refs = scrape_site.ReferenceData(slate_id, scoring_type_id)

draft_detail = scrape_site.DraftsDetail(draft_id, bearer_token)

df_players = refs.create_df_players_master()
df_draft = draft_detail.create_df_drafts()